# Python + SQL: MySQL Connector/Python

En esta lección vamos a aprender a conectarnos a bases de datos y realizar consultas SQL básicas desde un script de Python gracias al uso de MySQL Connector/Python.


## Conexión a una base de datos usando MySQL Connector:

A continuación vamos a ver cómo podemos conectarnos a una base de datos MySQL existente desde un script de Python usando *connect*.

El constructor de la clase, *connect()* crea una conexión al servidor MySQL y devuelve un objeto de tipo MySQLConnection. Veamos cómo usarlo mediante un ejemplo:


🚨🚨 **NOTA** 🚨🚨 En caso de que la celda de abajo os de error deberéis escribir en la terminal: 

```
pip install mysql-connector

y 

pip install mysql-connector-python
```

Para hacer esto fijaos que en la terminal os ponga "base" y que en el jupyter en la parte de arriba a la derecha ponga "base (y la versión de Python)". Después de esto, volved a ejecutar la celda. 

### Descripción de los argumentos de *connect()*

Antes de empezar, vamos a ver algunos de los argumentos principales que usaremos para la conexión a una base de datos usando el constructor *connect()*. 

- `user`: el nombre de usuario con el que nos queremos autenticar en el servidor MySQL. 

- `password`: la contraseña con la que nos queremos autenticar en el servidor MySQL. 

- `database`: el nombre de la base de datos a la que nos queremos conectar.

- `host`: el nombre o la dirección IP del servidor MySQL. El valor por defecto es "localhost" (o 127.0.0.1 que es lo mismo).

- `port`: el puerto TCP/IP del servidor MySQL. Debe ser un número entero. El valor por defecto es el puerto 3306.

Existen más argumentos que son opcionales o que se usarán cuando queramos configurar la conexión de una manera muy concreta. Podéis consultar la descripción de todos los parámetros [aquí](https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html). 

In [ ]:
import mysql.connector

In [ ]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='alumnaadalab',
                              host='127.0.0.1',
                              database='tienda')

print(cnx)
cnx.close()

En ocasiones podemos sufrir errores de conexión, por lo que es interesante añadir manejo de excepciones al código anterior. Usando *errorcode* podemos crear casos para cada tipo de error posible:

In [ ]:
# para añadir errores deberemos importarnos la siguiente librería
from mysql.connector import errorcode

# En este código estamos haciendo un try except. Si recordamos esto nos permitía hacer un manejo de los errores, para evitar que nuestro código se pare. Para eso lo que estamos haciendo es
## intenta hacer la conexión son la base de datos de tienda 
try:
  cnx = mysql.connector.connect(user='root', password='alumnaadalab',
                              host='127.0.0.1',
                              database='tienda')
# en caso de que no lo consigas por que hay algún error entonces ...
except mysql.connector.Error as err:

  # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  
  # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  
  # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
  else:
    print(err)
else:
  cnx.close()

Otra opción para pasar los argumentos de conexión a *connector()* es crear un diccionario que luego se pase como argumento al constructor:

In [ ]:
config = {
  'user': 'root',
  'password': 'alumnaadalab',
  'host': '127.0.0.1',
  'database': 'tienda',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()


## Desconexión de la base de datos usando MySQLConnector

Una vez hayamos terminado de realizar las consultas o trabajar de cualquier modo con la base de datos a la que nos hemos conectado con el connector, tendremos que desconectarnos usando el método *close()*:

In [ ]:

config = {
  'user': 'root',
  'password': 'alumnaadalab',
  'host': '127.0.0.1',
  'database': 'tienda',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cnx.close()

En este fragmente de código hemos creado la conexión cnx a la base de datos "tienda". Usando esa conexión podríamos por ejemplo realizar consultas sobre los datos almacenados en sus tablas (más adelante veremos como hacerlo). Una vez hayamos terminado y nos queramos desconectar será necesario usar el método *close()* del objeto connector que hemos definido (en este caso llamado cnx). Este método no recibe argumentos, por lo que su uso es sencillo.

## Realización de queries usando MySQLConnector:

Para realizar consultas a tablas de la base de datos a la que nos hemos conectado usando *connect()*, podemos usar el método *cursor()*. Empecemos viendo un ejemplo sencillo:

In [ ]:

# realizamos la conexión con la base de datos que queremos como hemos aprendido hasta ahora
cnx = mysql.connector.connect(user='root', password='alumnaadalab',
                              host='127.0.0.1',
                              database='tienda')

# Iniciamos el cursor y lo almacenamos en una variable llamada "cursor". Con esto estaremos creando un objeto de tipo cursor al que luego llamaremos para ejecutar las consultas SQL, 
## es decir, es el que nos va a permitir comunicarnos con la base de datos de SQL
cursor = cnx.cursor()

# El siguiente paso consiste en definir la consulta (query) que queremos realizar. 
## Para ello usamos todos los conocimientos sobre SQL que hemos adquirido durante el curso. La consulta se define como un string o cadena de texto.
### La consulta SQL de este ejemplo es sencilla y estática, pero podría incluir variables (lo veremos más adelante). 

query = ("""SELECT city, state FROM customers 
         WHERE customer_number BETWEEN 121 AND 124""")

# Luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
## De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
cursor.execute(query)

# es el momento de ver los resultados, como dijimos se almacenan en la variable "cursor". Para ello, usaremos un for loop para acceder a los resultados de una forma "humana". 

# Dependiendo del número de resultados que devuelva la consulta, será más interesante usar un método de acceso a los mismos u otro. 
## En el ejemplo anterior, hemos decidido usar el objeto cursor como un iterable e ir accediendo a cada registro mediante un bucle for. 
### Cada llamada al iterable de esta manera nos ha ido devolviendo un valor para cada atributo seleccionado en la consulta.
####  Más adelante veremos opciones alternativas para acceder a los resultados de las consultas.
for ciudad, estado in cursor:
  print(f"La ciudad {ciudad} pertenece al estado  {estado}")

cursor.close()
cnx.close()

Imaginemos ahora que nuestro jefe esta interesado en buscar el número de cliente, su estatus, la fecha en la que se realizó y el número de la factura de los pedidos que tenemos en nuestra base de datos. Pero no nos interesa para todas las fechas, solo queremos esta información para los pedidos realizados entre las siguientes fechas: 

- 1 Enero 2003
- 31 Marzo 2003. 

Para esto lo primero que vamos a necesitar son las fechas, pero para esto recordemos que SQL necesita un formato específico (AAAA-MM-DD). Podríamos pasar las fechas a mano, pero esto es muy poco eficaz. Para sacar las fechas en el formato que necesitamos usaremos la librería `datetime`

In [ ]:
import datetime

hire_start = datetime.date(2003, 1, 1)
hire_end = datetime.date(2003, 3, 31)

print(f"La fecha de inicio es {hire_start} y la de final es {hire_end}")

`datetime` es una librería suuuper útil cuando queremos trabajar con fechas. [Aquí](https://www.programiz.com/python-programming/datetime) os dejamos más documentación sobre esta librería.

Genial! Ya tenemos nuestras fechas en el formato que necesitamos. Ahora es el momento de ponernos a construir el código que necesitamos como aprendimos hasta ahora. 

In [ ]:

# realizamos la conexión con la BBDD que queremos
cnx = mysql.connector.connect(user='root', password='alumnaaadalab',
                              host='127.0.0.1',
                              database='tienda')

# iniciamos el cursor                               
cursor = cnx.cursor()

# escribimos la query con los datos que querremos extraer. Aquí, si nos fijamos hay una cosa que nos debe llamar la atención, y es la sintaxis que usamos en el WHERE, en conreto el símbolo "%s". 
## esto nos servirá para hacer queries dinámicas. En este caso, al poner estos símbolos lo que hacemos es decirle a SQL que será un valor que puede cambiar. ¿Nuestra duda ahora puede ser, como le pasamos esos valores?
### esto lo haremos cuando ejecutemos en el comando execute, como vemos en la siguiente línea. 
query = ("SELECT customer_number, status, order_date, order_number FROM orders "
         "WHERE order_date BETWEEN %s AND %s")

# si nos fijamos aquí la cosa ha cambiado un poco respecto al ejemplo anterior, y es que vemos que hemos añadido una tupla con las variables de las fechas sobre las que queremos buscar. Así es como especificaremos cuales
## son los valores específicos de nuestra búsqueda.  
cursor.execute(query, (hire_start, hire_end))

# por último mostramos los resultados de la misma forma que en el ejemplo anterior. 
for customer_number, status, order_date, order_number in cursor:
  print(f"El numero de orden {order_number}, esta en estado {status} con fecha {order_date}")

cursor.close()
cnx.close()


Cómo detalle, hasta ahora al `cursor` no le hemos pasado ningún parámetro, pero lo podemos hacer. Los argumentos que podemos usar para inicializarlo y algunos métodos útiles que podemos llamar desde nuestro código Python:

```python
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='tienda')
cursor = cnx.cursor([arg=value[, arg=value]...])
```

Los argumentos que se pueden pasar al método *cursor()* son:

- **buffered**: si es True, el cursor recupera (fetch) todas las filas de la base de datos cuando se ejecuta la query. Este tipo de cursors es útil cuando las consultas devuelven un número pequeño de resultados, o cuando queremos combinar el resultado de múltiples queries. En el caso de que lo configuremos como False, las filas de la base de datos se irán recuperando según se vayan solicitando (por ejemplo cuando usamos el cursor como un iterable). De esta manera no llenaremos la memoria de la máquina cuando las consultas devuelvan un número muy elevado de registros.
 
- **raw**: si lo configuramos como True, el cursor no realizará las conversiones automáticas entre tipos de datos de MySQL y Python (como sí hacía en el caso de las fechas en los ejemplos que hemos visto anteriormente). Un cursor de tipo raw es interesante cuando queremos máxima velocidad o cuando queremos hacer alguna otra conversión diferente a la por defecto.

- **dictionary**: si lo configuramos como True, el cursor devolverá las filas de los resultados como diccionarios. 

- **named_tuple**: si lo configuramos como True, el cursor devolverá las filas de los resultados como named tuples (tuplas con un nombre por el que podemos llamarlas para recuperar su contenido).

- **prepared**: si este argumento tiene el valor True, el cursor se configurará para ejecutar sentencias “prepared”.<!--¿SIGNIFICADO DE ESTO?-->

- **cursor_class**: un argumento que se puede usar para indicar que subclase queremos usar para instanciar el nuevo cursor. Hay varias clases que heredan de MySQLCursor como MySQLCursor Buffered, MySQLCursorRaw, etc. Dependiendo de la combinación de argumentos que pasemos a *cursor()* crearemos un cursor de una u otra de esas clases. Sin embargo, la clase final puede forzarse usando este argumento.


## Conocer las bases de datos del servidor

Si queremos conocer qué bases de datos existen en un servidor concreto, podemos hacerlo con el comando `SHOW DATABASES`, veamos como hacerlo:

In [ ]:
# conectamos con la BBDD
mydb = mysql.connector.connect(
                  user='root', password='alumnaadalab',
                  host='127.0.0.1',
                  database='tienda')
# iniciamos el cursor
mycursor = mydb.cursor()

# ejecutamos nuestra query para que nos muestre las BBDD que tenemos en nuestro servidor
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)


Normalmente en todos los servidores MySQL tendremos las bases de datos que hemos creado, y adicionalmente otras bases de datos adicionales llamadas 'mysql', 'sys' 'information_schema' y 'performance_schema'. Esas bases de datos contienen tablas con información acerca del propio servidor, las otras bases de datos y sus tablas, etc. En la siguiente sección por ejemplo usaremos 'information_schema' para conocer las tablas que componen una base de datos concreta.

## Conocer las tablas que componen una base de datos:

También podemos mostrar por pantalla los nombres de las tablas de una de las bases de datos del servidor. Para ello se usará la sentencia` SHOW TABLES`. De esta manera conoceremos mejor la estructura de la base de datos y si las tablas que queremos consultar se encuentran en ella.

In [ ]:
# conectamos con el servidor
mydb = mysql.connector.connect(
              user='root', password='alumnaadalab',
              host='127.0.0.1',
              database='tienda'
)

# creamos el servidor
mycursor = mydb.cursor()

# ejecutamos la query para ver las tablas de la BBDD que hemos indicado en la conexión
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

## Conocer las columnas que componen una tabla:

También podemos mostrar por pantalla la información de las columnas que componen una tabla específica. Podremos realizar esta tarea gracias a la existencia de la base de datos information_schema, cuyo objetivo es contener información acerca de el resto de las bases de datos del servidor y de sus tablas. La consulta que tendremos que realizar será la siguiente:

In [ ]:

# conectamos con el servidor
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="alumnaadalab",
  database="INFORMATION_SCHEMA" # en este caso tendremos que especificar que queremos información del esquema
)

# iniciamos el cursor
mycursor = mydb.cursor()

## ejecutamos nuestra query, especificando de que tabla queremos la información
mycursor.execute("SELECT * FROM COLUMNS WHERE TABLE_NAME = 'payments'")

for x in mycursor:
  print(x)
  break



## Modos de acceso a los resultados de la consulta

Como vimos al inicio de la lección el uso de *execute()* hace que el cursor se convierta en un iterable al cual podemos acceder de diferentes maneras. En ejemplos anteriores hemos visto como podemos acceder a cada uno de los elemenos del mismo como si fuese una lista mediante un bucle *for*. Sin embargo, MySQL Connector/Python nos proporciona maneras específicas para realizarlo.

### fetchone()

Si lo que queremos es acceder a la primera fila del resultado, se puede utilizar el método *fetchone()*. Este método devolverá la primera fila del resultado de la consulta y avanzará el cursor al siguiente registro del resultado. Debido a esto, la siguiente vez que ejecutemos el método *fetchone()* en el mismo cursor, se mostrarán diferentes datos. Veámoslo con un ejemplo:


In [ ]:
# conectamos con el servidor
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="alumnaadalab",
  database="tienda"
)

# iniciamos el cursor
mycursor = mydb.cursor()

# ejecutamos nuestra query
mycursor.execute("SELECT * FROM employees")

# en este caso solo nos interesa el primer resultado de nuestra query, por lo que usaremos el método fetchone, para que python solo nos devuelva el primero y después printemos el resultado
myresult = mycursor.fetchone()
print(myresult)

# volvemos a ejecutar el método fetchone, como ya nos mostró el primer resultado, y no hemos cerrado la conexión, al volver a ejecutarlo, nos mostrará el segundo resultado. Pero si nos fijamos, lo hace de uno en uno. 
myresult = mycursor.fetchone()
print(myresult)

### fetchall()

Si por el contrario lo que queremos es seleccionar todos los resultados que ha devuelto una consulta SQL, podemos usar el método *fetchall()*: 

In [ ]:
# establecemos la conexión con el servidor
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="alumnaadalab",
  database="tienda"
)

# inciamos el cursor
mycursor = mydb.cursor()

# realizamos nuestra query
mycursor.execute("SELECT * FROM employees")

# le decimos a Python que nos devuelva todos los resultados de la query usando el método fetchall
myresult = mycursor.fetchall()
print(myresult)

Si después quisieramos acceder a cada registro de los resultados, podemos hacerlo usando myresult como un iterable. Esto nos devolverá cada fila resultado como una tupla:

In [ ]:
# conectamos con el servidor
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="alumnaadalab",
  database="tienda"
)

# iniciamos el cursor
mycursor = mydb.cursor()

# realizamos la query
mycursor.execute("SELECT * FROM employees")

# le pedimos que nos muestre todos los resultados
myresult = mycursor.fetchall()

# accedemos a cada fila de una en una usando un bucle for. Fijaos que nos devuelve tuplas!!!
for x in myresult:
  print(x)

MySQL Connector/Python permite usar todo tipo de consultas SQL como argumento del método *execute()*. En los ejercicios finales vas a practicar su uso con operadores de SQL tales como WHERE, GROUP BY, HAVING, ORDER BY, LIMIT, etc.

## Integración de resultados de MySQL Connector/Python con Pandas

**¿Qué es Pandas?**

Pandas es una librería para Python que es ampliamente utilizada para el manejo de datos a media/gran escala. Cuando estamos trabajando con datos estructurados en forma de tabla como los almacenados en una base de datos SQL, una hoja de cálculo tipo Excel, etc., podemos usar Pandas para cargar esos datos, limpiarlos, procesarlos para extraer información y combinar diferentes fuentes de datos, entre otras cosas.

En Pandas, la estructura básica en las que cargaremos los datos se llama DataFrame. Una vez hemos creado el DataFrame con las fuentes de datos de nuestra elección (csv, excel, sql, json, etc.) podemos utilizar métodos ya incluidos en la librería para calcular la media, la mediana, el máximo o el mínimo de una columna numérica, entre otros.También podemos combinar los valores de varias columnas en una sola, agruparlas por categorías, en incluso realizar gráficos de manera sencilla gracias a la integración de Pandas con Matplotlib.

En esta parte de la lección veremos como cargar datos de una base de datos MySQL a un DataFrame de Pandas y haremos algunos cálculos sencillos sobre esos datos. En posteriores lecciones veremos una introducción más profunda a Pandas y a cómo usarlo. El objetivo de los siguientes ejemplos es simplemente que os quede claro cómo integrar MySQL Connector/Python con Pandas.


## Crear un dataframe de Pandas a partir de una sentencia SQL:

Como hemos indicado anteriormente, Pandas soporta la integración sencilla con muchos formatos de archivo y fuentes de datos: ficheros csv, excel y json, bases de datos sql, etc. Por ejemplo, para leer los datos de una base de datos SQL podemos ejecutar el siguiente trozo de código, el cual incluye una llamada al método constructor de DataFrames: 

In [3]:
import pandas as pd


In [ ]:
# importamos pandas
import pandas as pd

# hacemos la conexión con el servidor
cnx = mysql.connector.connect(user='root', password='alumnaadalab',
                              host='127.0.0.1',
                              database='tienda')

# iniciamos el cursor
mycursor = cnx.cursor()

# ejecutamos nuestra query
mycursor.execute("SELECT * FROM employees")

# le decimos que nos devuelva todos los resultados y los almacenamos en una variable llamada myresult
myresult = mycursor.fetchall()

#Creamos un dataframe con los resultados de la consulta SQL almacenados en myresult. Si os fijais le estamos pasando un parámetro llamado "columns" donde estamos especificando cuáles son las columnas de lo que será nuestro dataframe
df = pd.DataFrame(myresult, columns = ['ID', 'Nombre', 'Apellido','Email','Telefono','Direccion','Ciudad','Pais'])

#Cerramos la conexion
cnx.close()

In [ ]:
# mostramos las 5 primeras filas del dataframe usando el método .head()
df.head()

Otra opción más directa para ver los resultados de nuestra query en un DataFrame consiste en encargarle a Pandas directamente la ejecución de la consulta SQL. Para ello usaremos el método *read_sql_query()* como vemos a continuación: 

In [ ]:

# realizamos la conexión con el servidor
cnx = mysql.connector.connect(user='root', password='alumnaadalab',
                              host='127.0.0.1',
                              database='tienda')


# escribimos nuestra query
sql = "SELECT * FROM employees"

# utilizamos el método pd.read_sql_qury() para convertir los resultados de nuestra query en un DataFrame que podamos ver de forma amigable en Python
df = pd.read_sql_query(sql, cnx)

#Lo siguiente es un wrap up que funciona tanto con una tabla como con una consulta SQL.
#pd.read_sql(sql, cnx)

#Cerramos la conexion
cnx.close()

In [ ]:
# mostramos las 10 primeras filas de nuestros resultados 
df.head(10)


El método *read_sql_query()* necesita como argumentos un string de texto que contenga la consulta SQL que queramos realizar, y una variable que contenga la conexión con la base de datos sobre la que se vaya a ejecutar la consulta (en este caso la hemos obtenido con el método *connect()* de MySQL Connector pero acepta otras opciones). El resultado es el mismo que cuando usamos *myresult* como argumento del constructor de DataFrame, con la ventaja de que en este caso los nombres de las columnas del dataframe se toman automáticamente de los nombres de las columnas del resultado de la consulta SQL.

## Guardado de datos:

Una vez hayamos obtenido los resultados que queremos mediante el procesado de los datos con Pandas, esta librería también proporciona diferentes maneras de almacenar los datos gracias a el uso de los métodos *to_\** (cuyo nombre variará en función del formato en el que se quieren guardar los datos). Veamos unos ejemplos al respecto:

In [ ]:
# En este ejemplo calculabamos el valor más frecuente para el pais de las alumnas. 
# Ahora lo almacenaremos de diferentes formas.

#Creamos la conexion con el servidor
cnx = mysql.connector.connect(user='root', password='admin',
                              host='127.0.0.1',
                              database='tienda')

#Realizamos la consulta a la tabla alumnas mediante pandas
sql = "SELECT * FROM employees"
df = pd.read_sql_query(sql, cnx)


#Guardamos el dato en un fichero csv (separado por comas)
df.to_csv("fichero.csv")

#Formateamos el dato a un tipo string para facilitar la lectura en la consola
output_string = df.to_string()
print(output_string)

#Formateamos el dato a un tipo string para facilitar la inserción de los datos en un documento latex (el formato usado más comunmente en la escritura de artículos científicos)
output_latex = df.to_latex()
print(output_latex)


#Cerramos la conexion
cnx.close()

En el código anterior hemos visto cómo guardar los datos obtenidos con Pandas en diferentes formatos: un fichero .csv para su posible procesado en un futuro, un string para su visualización por pantalla y un formateo estilo latex para su uso en documentos científicos. Cabe resaltar que estos son solo tres ejemplos sencillos, pero que en la documentación de Pandas podéis encontrar muchos más ejemplos acerca de los formatos en los que se pueden guardar los datos: [Aquí](https://pandas.pydata.org/docs/reference/#) podréis encontrar más documentación.  


#### ENUNCIADO EJERCICIOS

En este conjunto de ejercicios vamos a volver a usar la tabla Customers (Clientes/as) que vamos a importar en MySQL Workbench. Si tienes dudas de como importarla, revisita la página asociada de tutorial.

La tabla Customers tiene las siguientes columnas:

**Customers**(`customerNumber`, `customerName`, `contactLastName`, `contactFirstName`, `phone`, `addressLine1`, `addressLine2`, `city`, `state`, `postalCode`, `country`, `salesRepEmployeeNumber`, `creditLimit`) 

Cada columna es bastante autodescriptiva en su nombre, pero vamos a incluir una pequeña descripción: 

- *customerNumber*: el número identificativo de las clientas/es. Es un número entero y sirve de clave primaria.
- *customerName*: el nombre de las empresas en las que trabajan las/los clientas/es. Es una cadena de texto.
- *contactLastName*: El apellido de la persona de contacto en la empresa cliente. Es una cadena de texto.
- *contactFirstName*: El nombre de la persona de contacto en la empresa cliente. Es una cadena de texto.
- *phone*: El teléfono de la persona de contacto en la empresa cliente. Es una cadena de texto (ya que hay espacios).
- *adressLine1*: La dirección (calle, número, etc.) de la empresa cliente. Es una cadena de texto.
- *adressLine2*: La dirección de la empresa cliente (si se necesita mas espacio). Es una cadena de texto. Muchas veces está vacía.
- *city*: La ciudad de la empresa cliente.
- *state*: El estado en el que se encuentra la empresa cliente. Válido para los Estados Unidos. Es una cadena de texto.
- *postalCode*: El código postal. Es una cadena de texto (ya que puede haber espacios).
- *country*: El país de la empresa cliente. Es una cadena de texto.
- *salesRepEmployeeNumber*: El número identificador de la empleada o empleado que lleva a esa empresa cliente. Es un número entero.
- *creditLimit*: El límite de crédito que tiene la empresa cliente. 

La tabla Employees tiene las siguientes columnas: 

- *employeeNumber*: el número identificativo de las empleadas/os. Es un número entero y sirve de clave primaria.
- *lastName*: el apellido de las empleadas. Es una cadena de texto.
- *firstName*: el nombre de las empleadas. Es una cadena de texto.
- *extension*: su extensión telefónica. Es una cadena de texto.
- *email*: el correo electrónico de la empleada. Es una cadena de texto.
- *officeCode*: El código de la oficina de la empleada. Es una cadena de texto.
- *reportsTo*: el número identificativo de la empleada a la que reporta (su supervisora). Es un número entero y clave foránea (relacionada con employeeNumber).
- *jobTitle*: el nombre del puesto de trabajo que desempeña. Es una cadena de texto.



#### EJERCICIO 1

Imprime por pantalla el nombre de las tablas que componen la base de datos "tienda".

#### EJERCICIO 2

Imprime por pantalla la información acerca de las columnas que componen la tabla "Employees":

#### EJERCICIO 3

Realiza una consulta SELECT que obtenga los nombres, teléfonos y direcciones de todas las empresas cliente de la tabla customers. Imprime por pantalla los datos recuperados usando el método *fetchone()*

#### EJERCICIO 4

Buscar aquellos registros de la tabla Customers que correspondan a clientes de USA pero que no tengan un valor guardado para el campo *state*. Accede a los datos con *fetchall()*

#### EJERCICIO 5

Realiza una consulta SELECT que obtenga el número identificativo de cliente más bajo de la base de datos.


#### EJERCICIO 6

Selecciona el máximo de credito que tiene cualquiera de los clientes del empleado con número 1165.

#### EJERCICIO 7

Selecciona el número de clientes en cada pais.

#### EJERCICIO 8

Crea un dataframe que contenga todas las entradas de la tabla Customers. Para ello usa un *cursor* y el método *execute()*:

#### EJERCICIO 9

Como Customers tiene un número de registros muy elevado, crea un nuevo dataframe.

#### EJERCICIO 10

El resultado anterior tiene 13 columnas, por lo que no es cómodo para su visualización por el terminal (o en este mismo notebook).
Crea un dataframe con los resultados. 